In [1]:
import os
import io
import math
import time
import ssl
import certifi
import urllib3
import itertools
from collections import Counter
from statistics import correlation
from importlib import import_module
from io import BytesIO

# Data Handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import matplotlib.cm as cm
import seaborn as sns

# Statistics & Math
from scipy.stats import t, norm
from scipy.stats.mstats import winsorize

# Machine Learning
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import linear_model
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import GradientBoostingRegressor

# Time Series
from pandas.tseries.offsets import DateOffset

# File Handling & Parquet
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

# APIs & Data Sources
import requests
import wrds
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random

In [2]:
folder = os.path.expanduser('~/Desktop/Machine Learning/data_ML_Project')
folder1 = os.path.expanduser('~/Desktop/Machine Learning/data_filtered')

# CSV Files
main_csv = os.path.join(folder, 'monthly_crsp.csv')
jkp_csv = os.path.join(folder1, 'JKP_filtered.csv')
zimmer_csv = os.path.join(folder1, 'Zimmer_filtered.csv')


if os.path.exists(folder):
    data_main = pd.read_csv(main_csv)
    data_jkp = pd.read_csv(jkp_csv)
    data_zimmer = pd.read_csv(zimmer_csv)
    
else:
    print("File still not found!")


In [3]:
data_main.head()

,PERMNO,HdrCUSIP,CUSIP,Ticker,TradingSymbol,PERMCO,SICCD,NAICS,MthCalDt,MthRet,sprtrn
0,10000,68391610,68391610,OMFGA,OMFGA,7952,3990,0,1986-01-31,0.707317,0.002367
1,10000,68391610,68391610,OMFGA,OMFGA,7952,3990,0,1986-02-28,-0.257143,0.071489
2,10000,68391610,68391610,OMFGA,OMFGA,7952,3990,0,1986-03-31,0.365385,0.052794
3,10000,68391610,68391610,OMFGA,OMFGA,7952,3990,0,1986-04-30,-0.098592,-0.014148
4,10000,68391610,68391610,OMFGA,OMFGA,7952,3990,0,1986-05-30,-0.222656,0.050229


In [4]:
# Ensure 'date' is in datetime format
data_main['date'] = pd.to_datetime(data_main['MthCalDt'])
data_jkp['date'] = pd.to_datetime(data_jkp['date'])
data_zimmer['date'] = pd.to_datetime(data_zimmer['date'], errors='coerce')

# Filter from 1986 onwards
start_date = pd.to_datetime('1986-01-01')
data_main = data_main[data_main['date'] >= start_date]
data_jkp = data_jkp[data_jkp['date'] >= start_date]
data_zimmer = data_zimmer[data_zimmer['date'] >= start_date]

# Create 'year' and 'month' columns for merging
for df in [data_main, data_jkp, data_zimmer]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month

# Merge first with data_jkp
merged = data_main.merge(
    data_jkp.drop(columns=['date']),
    on=['year', 'month'],
    how='left',
    suffixes=('', '_jkp')
)

# Merge next with data_zimmer
merged = merged.merge(
    data_zimmer.drop(columns=['date']),
    on=['year', 'month'],
    how='left',
    suffixes=('', '_zimmer')
)


In [5]:
# Keep only tickers with at least 300 records
merged = merged[merged.groupby('Ticker')['Ticker'].transform('count') >= 300]


In [6]:
print("String columns:", merged.select_dtypes(include=["object", "string"]).columns.tolist())

String columns: ['HdrCUSIP', 'CUSIP', 'Ticker', 'TradingSymbol', 'MthCalDt']


In [7]:
columns_to_drop = [ 
    'PERMNO', 'HdrCUSIP', 'CUSIP', 'TradingSymbol', 'PERMCO', 'SICCD', 'NAICS','date','year','month'
]

merged.drop(columns=columns_to_drop, inplace=True)

In [8]:
# Shift 'MthRet' and 'sprtrn' within each Ticker group
merged[['MthRet', 'sprtrn']] = merged.groupby('Ticker')[['MthRet', 'sprtrn']].shift(1).fillna(0)

In [9]:
class FlexibleMLP(nn.Module):
    def __init__(self, layers: list, scale: float = 1.0, bias_scale: float = 0.0, activation=nn.GELU()):
        """
        Initialize a customizable Multi-Layer Perceptron (MLP) with flexible architecture and initialization.

        Args:
            layers (list of int): A list of integers where each value defines the size of each layer.
                                  For example, layers=[10, 64, 32, 1] defines a network with input dimension 10,
                                  two hidden layers with sizes 64 and 32, and output dimension 1.
            scale (float): Scaling factor for weight initialization. Controls the standard deviation of
                           the normal distribution used in initializing weights. Recommended to be 1.0 for LeCun initialization.
            bias_scale (float): Scaling factor for bias initialization. Often set to 0.0 to start with no initial bias.
            activation (nn.Module): Activation function applied after each linear transformation except the last layer.
                                    Defaults to nn.GELU(), but can be any activation like nn.ReLU(), nn.Tanh(), etc.
        """
        # Call the constructor of the parent class (nn.Module) to initialize all internal PyTorch machinery.
        # This is crucial because nn.Module handles a lot of behind-the-scenes logic like:
        # - registering parameters (weights and biases) for automatic optimization
        # - setting up .to(device), .eval(), .train(), etc.
        # - tracking submodules (layers, activations, etc.)
        #
        # If you omit this line, the module will NOT work correctly in PyTorch:
        # things like model.cuda(), model.parameters(), model.state_dict(), etc. will all break.
        #
        # The super() call here:
        # - FlexibleMLP is our class
        # - nn.Module is the parent class
        # - self.__init__() is the method we want to call from the parent
        super(FlexibleMLP, self).__init__()

        # Save arguments as attributes for reuse in reset_parameters
        self.layer_sizes = layers
        self.scale = scale
        self.bias_scale = bias_scale
        self.activation_fn = activation

        # Create containers to hold layers and activations
        self.layers = nn.ModuleList()
        self.activations = nn.ModuleList()

        # Build network structure (but not weights yet)
        self._build_layers()

        # Initialize weights and biases
        self.reset_parameters()

    def _build_layers(self):
        """
        Build the linear layers and corresponding activations (except for initialization).
        """
        for i in range(len(self.layer_sizes) - 1):
            # Create a linear layer from layer i to layer i+1
            layer = nn.Linear(self.layer_sizes[i], self.layer_sizes[i + 1])
            self.layers.append(layer)

            # Add an activation function unless it's the final layer
            if i < len(self.layer_sizes) - 2:
                self.activations.append(self.activation_fn)
            else:
                # Final layer doesn't use activation (use Identity to keep list structure consistent)
                self.activations.append(nn.Identity())

    def reset_parameters(self):
        """
        Apply custom initialization to all layers using the given scale and bias_scale.
        """
        for i, layer in enumerate(self.layers):
            # Apply LeCun-style initialization for better gradient behavior
            nn.init.normal_(layer.weight, mean=0.0, std=self.scale * np.sqrt(1 / self.layer_sizes[i]))
            nn.init.normal_(layer.bias, mean=0.0, std=self.bias_scale * np.sqrt(1 / self.layer_sizes[i]))

    def forward(self, x, return_last_hidden=False):
        """
        Perform a forward pass through the network.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_dim).
            return_last_hidden (bool): If True, returns both the final output and the last hidden layer's output.
                                       Useful for feature extraction, analysis, or interpretability.

        Returns:
            torch.Tensor: Output of the final layer.
            torch.Tensor (optional): Output of the last hidden layer (before final linear layer),
                                     if return_last_hidden is set to True.
        """
        last_hidden = None  # Will store the output of the last hidden layer

        # Apply all but the last layer with activation
        for layer, activation in zip(self.layers[:-1], self.activations[:-1]):
            x = activation(layer(x))  # Apply linear transformation and activation
            last_hidden = x  # Save the last hidden output

        # Final layer (linear transformation only, no activation)
        x = self.layers[-1](x)

        if return_last_hidden:
            return x, last_hidden
        return x

In [10]:
def set_seed(seed_value=42):
    """
    Set random seed across NumPy, Python, and PyTorch to ensure reproducibility.

    Args:
        seed_value (int): The seed value to use. Default is 42, a commonly used arbitrary number.

    This function ensures that experiments produce the same results across different runs,
    which is critical for debugging, comparing models, and scientific reproducibility.

    It sets the seed for:
        - NumPy (used for numerical ops like matrix generation)
        - Python's built-in random module (used in random sampling, shuffling, etc.)
        - PyTorch (both CPU and GPU)

    For GPU reproducibility:
        - It manually sets the CUDA seeds (for single and multi-GPU setups)
        - It disables the CUDA backend benchmarking feature to ensure deterministic behavior
          (at the potential cost of performance).
    """

    # Set seed for NumPy (used in data shuffling, batch generation, etc.)
    np.random.seed(seed_value)

    # Set seed for PyTorch operations on CPU
    torch.manual_seed(seed_value)

    # Set seed for Python's built-in random module (e.g., random.shuffle, random.randint)
    random.seed(seed_value)

    # Set seeds for PyTorch operations on GPU
    if torch.cuda.is_available():
        # Set seed for single-GPU
        torch.cuda.manual_seed(seed_value)

        # Set seed for all available GPUs (multi-GPU training)
        torch.cuda.manual_seed_all(seed_value)

        # Ensures that CUDA uses deterministic algorithms
        # This disables non-deterministic optimizations and ensures reproducible behavior
        torch.backends.cudnn.deterministic = True

        # Disables cuDNN auto-tuner which selects the best algorithm for each configuration
        # When disabled, it uses deterministic algorithms, but this might make training slower
        torch.backends.cudnn.benchmark = False
set_seed(42)

In [11]:
def train_model(num_epochs: int,
                train_loader: torch.utils.data.DataLoader,
                criterion,
                optimizer,
                model: torch.nn.Module,
                ridge_penalty: float = 0.001):
    """
    Train a PyTorch model using mini-batch gradient descent with optional L2 (ridge) regularization.

    Args:
        num_epochs (int): Number of full passes through the training dataset.
        train_loader (DataLoader): PyTorch DataLoader containing training data in mini-batches.
        criterion: Loss function (e.g., nn.MSELoss).
        optimizer: PyTorch optimizer (e.g., torch.optim.Adam or SGD).
        model (nn.Module): The model to train.
        ridge_penalty (float): L2 regularization strength (default: 0.001). Helps reduce overfitting.

    Returns:
        None. Prints training loss every 20 epochs.
    """
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            # Forward pass: get model predictions for this batch
            outputs = model(inputs)

            # Compute loss = prediction error + ridge penalty (L2 regularization)
            # The penalty term prevents overfitting by discouraging large weights
            # sum(p.abs().pow(2).sum()) is equivalent to ||W||² (squared L2 norm)
            loss = criterion(outputs, targets) + ridge_penalty * sum(p.pow(2.0).sum() for p in model.parameters())

            # Backward pass and optimization:
            optimizer.zero_grad()   # Clear previous gradients to prevent accumulation
            loss.backward()         # Backpropagate the current loss
            optimizer.step()        # Perform a parameter update (gradient descent step)

        # Print progress every 20 epochs
        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [39]:
def predict_returns_for_ticker(ticker_data, ticker, epochs=50, batch_size=32, num_seeds=10):
    # Prepare features and target
    X = ticker_data.drop(columns=['MthCalDt', 'sprtrn', 'Ticker', 'MthRet']).fillna(0)
    y = ticker_data['MthRet'].fillna(0)
    dates = ticker_data['MthCalDt']
    sprtrn = ticker_data['sprtrn'].fillna(0)

    # Split data
    X_train, X_test, y_train, y_test, dates_train, dates_test, sprtrn_train, sprtrn_test = train_test_split(
        X, y, dates, sprtrn, test_size=0.2, random_state=42, shuffle=False
    )
    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

    # Create data loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Initialize model and train with different seeds
    input_size = X_train.shape[1]
    train_predictions_all = []
    test_predictions_all = []
    for seed in range(num_seeds):
        # Set random seed for reproducibility
        torch.manual_seed(seed)
        np.random.seed(seed)

        # Initialize model
        model = FlexibleMLP(layers=[input_size, 64, 32, 1])
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # Train model
        train_model(
            num_epochs=epochs,
            train_loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            model=model,
            ridge_penalty=0.001
        )

        # Generate predictions
        model.eval()
        with torch.no_grad():
            train_predictions = model(X_train_tensor).squeeze().numpy()
            test_predictions = model(X_test_tensor).squeeze().numpy()
            # Handle single-sample case
            if X_test.shape[0] == 1:
                test_predictions = np.array([test_predictions])
            train_predictions_all.append(train_predictions)
            test_predictions_all.append(test_predictions)

    # Convert predictions to arrays with shape (n_samples, num_seeds)
    try:
        train_predictions_all = np.stack(train_predictions_all, axis=1)  # Shape: (n_train_samples, 10)
        test_predictions_all = np.stack(test_predictions_all, axis=1)    # Shape: (n_test_samples, 10)
    except ValueError as e:
        print(f"Ticker {ticker}: Error stacking predictions: {e}")
        raise

    # Average predictions across seeds
    train_predictions_avg = train_predictions_all.mean(axis=1)
    test_predictions_avg = test_predictions_all.mean(axis=1)

    # Combine train and test predictions
    all_predictions = np.concatenate([train_predictions_avg, test_predictions_avg])
    all_actual = np.concatenate([y_train.values, y_test.values])
    all_dates = pd.concat([dates_train.reset_index(drop=True), dates_test.reset_index(drop=True)])

    # Create DataFrame for predictions
    predictions_df = pd.DataFrame({
        'Date': all_dates,
        'Actual_MthRet': all_actual,
        'Predicted_MthRet': all_predictions
    })

    # Compute timed returns and market comparison using mean predictions
    test_targets = y_test.values
    test_dates = dates_test
    sprtrn = sprtrn_test.values

    # Ensure shapes are correct
    timed_returns = test_targets * test_predictions_avg  # Shape: (n_test_samples,)
    together = pd.DataFrame({
        'Actual_MthRet': test_targets,
        'Timed_Returns': timed_returns,
        'Market_Returns': sprtrn
    }, index=test_dates)

    def sharpe_ratio(returns):
        return np.mean(returns) / np.std(returns) * np.sqrt(12) if np.std(returns) != 0 else 0

    sharpe_ratios = together.apply(sharpe_ratio)
    cumulative_returns = together.cumsum()

    market_sharpe = sharpe_ratios['Market_Returns']
    market_cumulative = cumulative_returns['Market_Returns'].iloc[-1]

    performance_summary = pd.DataFrame({
        'Metric': ['Sharpe_Ratio', 'Cumulative_Return', 'Beats_Market_Sharpe', 'Beats_Market_Cumulative'],
        'Value': [
            sharpe_ratios['Timed_Returns'],
            cumulative_returns['Timed_Returns'].iloc[-1],
            sharpe_ratios['Timed_Returns'] > market_sharpe,
            cumulative_returns['Timed_Returns'].iloc[-1] > market_cumulative
        ]
    })

    (together / together.std()).cumsum().plot(figsize=(10, 6))
    plt.title(f'{ticker} Model vs Market (Market Sharpe: {market_sharpe:.2f})')
    plt.xlabel('Date')
    plt.ylabel('Standardized Cumulative Returns')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f'{ticker}_cumulative_returns.png'))
    plt.close()

    # Save performance summary and predictions
    performance_summary.to_csv(os.path.join(output_dir, f'{ticker}_performance_summary.csv'), index=False)
    predictions_df.to_csv(os.path.join(output_dir, f'{ticker}_predictions.csv'), index=False)

    return {
        'ticker': ticker,
        'mse': mean_squared_error(test_targets, test_predictions_avg),
        'performance_summary': performance_summary,
        'predictions_df': predictions_df
    }

In [41]:
output_dir = os.path.expanduser('~/Desktop/Machine Learning/predictions')
os.makedirs(output_dir, exist_ok=True)

In [42]:
# Run predictions for all tickers
tickers = merged['Ticker'].unique()
results = []
for ticker in tqdm(tickers, desc="Predicting returns for tickers"):
    ticker_data = merged[merged['Ticker'] == ticker].copy()
    if len(ticker_data) < 50:  # Skip tickers with insufficient data
        continue
    try:
        result = predict_returns_for_ticker(ticker_data, ticker)
        results.append(result)
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")

# Aggregate results
results_df = pd.DataFrame([
    {'ticker': r['ticker'], 'mse': r['mse']}
    for r in results
])

Predicting returns for tickers:   0%|          | 0/2694 [00:00<?, ?it/s]

Epoch [20/50], Loss: 0.0887
Epoch [40/50], Loss: 0.0774
Epoch [20/50], Loss: 0.0853
Epoch [40/50], Loss: 0.0688
Epoch [20/50], Loss: 0.0786
Epoch [40/50], Loss: 0.0552
Epoch [20/50], Loss: 0.1005
Epoch [40/50], Loss: 0.0697
Epoch [20/50], Loss: 0.0802
Epoch [40/50], Loss: 0.0644
Epoch [20/50], Loss: 0.0817
Epoch [40/50], Loss: 0.0573
Epoch [20/50], Loss: 0.0957
Epoch [40/50], Loss: 0.0743
Epoch [20/50], Loss: 0.0952
Epoch [40/50], Loss: 0.0730
Epoch [20/50], Loss: 0.0829
Epoch [40/50], Loss: 0.0630


Predicting returns for tickers:   0%|          | 1/2694 [00:02<1:44:41,  2.33s/it]

Epoch [20/50], Loss: 0.0914
Epoch [40/50], Loss: 0.0751
Epoch [20/50], Loss: 0.0821
Epoch [40/50], Loss: 0.0664
Epoch [20/50], Loss: 0.0860
Epoch [40/50], Loss: 0.0677
Epoch [20/50], Loss: 0.0717
Epoch [40/50], Loss: 0.0447
Epoch [20/50], Loss: 0.0838
Epoch [40/50], Loss: 0.0685
Epoch [20/50], Loss: 0.0747
Epoch [40/50], Loss: 0.0524
Epoch [20/50], Loss: 0.0793
Epoch [40/50], Loss: 0.0560
Epoch [20/50], Loss: 0.0882
Epoch [40/50], Loss: 0.0712
Epoch [20/50], Loss: 0.0874
Epoch [40/50], Loss: 0.0699
Epoch [20/50], Loss: 0.0806
Epoch [40/50], Loss: 0.0590


Predicting returns for tickers:   0%|          | 2/2694 [00:04<1:42:39,  2.29s/it]

...

Epoch [20/50], Loss: 0.0840
Epoch [40/50], Loss: 0.0679
Epoch [20/50], Loss: 0.0832
Epoch [40/50], Loss: 0.0667
Epoch [20/50], Loss: 0.0832
Epoch [40/50], Loss: 0.0666
Epoch [20/50], Loss: 0.0667
Epoch [40/50], Loss: 0.0388
Epoch [20/50], Loss: 0.0776
Epoch [40/50], Loss: 0.0570
Epoch [20/50], Loss: 0.0718
Epoch [40/50], Loss: 0.0462
Epoch [20/50], Loss: 0.0713
Epoch [40/50], Loss: 0.0459
Epoch [20/50], Loss: 0.0819
Epoch [40/50], Loss: 0.0625
Epoch [20/50], Loss: 0.0826
Epoch [40/50], Loss: 0.0651
Epoch [20/50], Loss: 0.0721
Epoch [40/50], Loss: 0.0470


Predicting returns for tickers: 100%|█████████▉| 2693/2694 [1:46:04<00:02,  2.36s/it]

Epoch [20/50], Loss: 0.0884
Epoch [40/50], Loss: 0.0687
Epoch [20/50], Loss: 0.0864
Epoch [40/50], Loss: 0.0700
Epoch [20/50], Loss: 0.0837
Epoch [40/50], Loss: 0.0655
Epoch [20/50], Loss: 0.0724
Epoch [40/50], Loss: 0.0461
Epoch [20/50], Loss: 0.0852
Epoch [40/50], Loss: 0.0641
Epoch [20/50], Loss: 0.0732
Epoch [40/50], Loss: 0.0483
Epoch [20/50], Loss: 0.0824
Epoch [40/50], Loss: 0.0589
Epoch [20/50], Loss: 0.0873
Epoch [40/50], Loss: 0.0719
Epoch [20/50], Loss: 0.0857
Epoch [40/50], Loss: 0.0720
Epoch [20/50], Loss: 0.0836
Epoch [40/50], Loss: 0.0587
Epoch [20/50], Loss: 0.0886


Predicting returns for tickers: 100%|██████████| 2694/2694 [1:46:06<00:00,  2.36s/it]

Epoch [40/50], Loss: 0.0742


In [43]:
# Summary statistics
print("\nSummary Statistics for Test Set MSE:")
print(results_df['mse'].describe())


Summary Statistics for Test Set MSE:
count    2694.000000
mean        0.026826
std         0.093897
min         0.000198
25%         0.007639
50%         0.014350
75%         0.028434
max         3.308881
Name: mse, dtype: float64
